In [1]:
from keras.applications.xception import Xception
from keras.preprocessing import image
from keras.applications.xception import preprocess_input
import numpy as np
import pandas as pd
from keras.layers import Input, Dense, Dropout, Flatten
from keras.models import Model, Sequential
from keras.optimizers import SGD
import keras.backend as K

Using TensorFlow backend.


In [10]:
def precision(y_true, y_pred):
    """Precision metric.

    Only computes a batch-wise average of precision.

    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

In [11]:
def recall(y_true, y_pred):
    """Recall metric.

    Only computes a batch-wise average of recall.

    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

In [12]:
def fscore(y_true, y_pred, beta=1):
    """Computes the F score.

    The F score is the weighted harmonic mean of precision and recall.
    Here it is only computed as a batch-wise average, not globally.

    This is useful for multi-label classification, where input samples can be
    classified as sets of labels. By only using accuracy (precision) a model
    would achieve a perfect score by simply assigning every class to every
    input. In order to avoid this, a metric should penalize incorrect class
    assignments as well (recall). The F-beta score (ranged from 0.0 to 1.0)
    computes this, as a weighted mean of the proportion of correct class
    assignments vs. the proportion of incorrect class assignments.

    With beta = 1, this is equivalent to a F-measure. With beta < 1, assigning
    correct classes becomes more important, and with beta > 1 the metric is
    instead weighted towards penalizing incorrect class assignments.
    """
    if beta < 0:
        raise ValueError('The lowest choosable beta is zero (only precision).')

    # If there are no true positives, fix the F score at 0 like sklearn.
    if K.sum(K.round(K.clip(y_true, 0, 1))) == 0:
        return 0

    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    bb = beta ** 2
    fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon())
    return fbeta_score

In [3]:
model = Xception(weights='imagenet', include_top=False)

In [2]:
datagen = image.ImageDataGenerator()

In [4]:
train_generator = datagen.flow_from_directory('image_data/train',
                                              target_size=(299,299),
                                              batch_size=32,
                                              class_mode='binary')

Found 9301 images belonging to 2 classes.


In [5]:
test_generator = datagen.flow_from_directory('image_data/test',
                                             target_size=(299,299),
                                             batch_size=32,
                                             class_mode='binary')

Found 2326 images belonging to 2 classes.


In [6]:
top_model = Sequential()
top_model.add(Flatten(input_shape=(10,10,2048)))
top_model.add(Dense(1024, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1024, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

In [7]:
for l in model.layers:
    l.called_with = None

In [8]:
inputs = Input(shape=(299,299,3))
x = model(inputs)
preds = top_model(x)
combined_model = Model(input=inputs, output=preds)

In [9]:
for l in combined_model.layers[1].layers:
    l.trainable = False

In [13]:
combined_model.compile(loss='mse',
                       optimizer='sgd',
                       metrics=['accuracy', precision, recall, fscore])

In [14]:
combined_model.fit_generator(train_generator,
                             samples_per_epoch=2048,
                             nb_epoch=16,
                             validation_data=test_generator,
                             nb_val_samples=512,
                             verbose=2)

Epoch 1/16
1048s - loss: 0.2461 - acc: 0.6372 - precision: 0.6406 - recall: 0.6029 - fscore: 0.5939 - val_loss: 0.2009 - val_acc: 0.6895 - val_precision: 0.6270 - val_recall: 0.9362 - val_fscore: 0.7464
Epoch 2/16
1040s - loss: 0.2143 - acc: 0.6958 - precision: 0.7068 - recall: 0.7367 - fscore: 0.7061 - val_loss: 0.1619 - val_acc: 0.7871 - val_precision: 0.8014 - val_recall: 0.7489 - val_fscore: 0.7633
Epoch 3/16
1035s - loss: 0.2074 - acc: 0.6968 - precision: 0.6962 - recall: 0.7142 - fscore: 0.6924 - val_loss: 0.1521 - val_acc: 0.7930 - val_precision: 0.7590 - val_recall: 0.8784 - val_fscore: 0.8106
Epoch 4/16
1039s - loss: 0.2034 - acc: 0.7129 - precision: 0.7144 - recall: 0.7521 - fscore: 0.7173 - val_loss: 0.1602 - val_acc: 0.7812 - val_precision: 0.7512 - val_recall: 0.8537 - val_fscore: 0.7921
Epoch 5/16


/home/cathyw/miniconda3/lib/python3.5/site-packages/keras/engine/training.py:1480: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


1047s - loss: 0.1844 - acc: 0.7424 - precision: 0.7346 - recall: 0.7946 - fscore: 0.7507 - val_loss: 0.1591 - val_acc: 0.7656 - val_precision: 0.7924 - val_recall: 0.7551 - val_fscore: 0.7692
Epoch 6/16
1044s - loss: 0.1714 - acc: 0.7471 - precision: 0.7381 - recall: 0.7665 - fscore: 0.7420 - val_loss: 0.1470 - val_acc: 0.7940 - val_precision: 0.8006 - val_recall: 0.7927 - val_fscore: 0.7923
Epoch 7/16
1035s - loss: 0.1710 - acc: 0.7622 - precision: 0.7612 - recall: 0.7917 - fscore: 0.7665 - val_loss: 0.1683 - val_acc: 0.7578 - val_precision: 0.7892 - val_recall: 0.7296 - val_fscore: 0.7499
Epoch 8/16
1035s - loss: 0.1780 - acc: 0.7466 - precision: 0.7302 - recall: 0.7661 - fscore: 0.7385 - val_loss: 0.1393 - val_acc: 0.8125 - val_precision: 0.7998 - val_recall: 0.8393 - val_fscore: 0.8168
Epoch 9/16
1042s - loss: 0.1646 - acc: 0.7686 - precision: 0.7663 - recall: 0.7957 - fscore: 0.7711 - val_loss: 0.1543 - val_acc: 0.7865 - val_precision: 0.7513 - val_recall: 0.8526 - val_fscore: 0.7